In [1]:
'''
0 --- Dovish
1 --- Hawkish
2 --- Neutral
'''

'\n0 --- Dovish\n1 --- Hawkish\n2 --- Neutral\n'

In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential

2023-12-06 23:27:09.842696: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#preprocess the data
def preprocess(text):
    max_words = 5000
    tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
    tokenizer.fit_on_texts(text)
    
    #convert text to sequences of indices
    sequences = tokenizer.texts_to_sequences(text)

    #padding sequences
    max_length = 100
    padded_sequence = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

    return np.array(padded_sequence)

In [4]:
#import lab-manual-combine-training data
combine_training_data = pd.DataFrame()

# Read an Excel file using pandas
combine_5768 = '/Users/simonli/Desktop/data297/fomc-hawkish-dovish-main/training_data/test-and-training/training_data/lab-manual-combine-train-5768.xlsx'
df_5768 = pd.read_excel(combine_5768)

combine_training_data = combine_training_data.append(df_5768, ignore_index=True)

combine_78516 = '/Users/simonli/Desktop/data297/fomc-hawkish-dovish-main/training_data/test-and-training/training_data/lab-manual-combine-train-78516.xlsx'
df_78516 = pd.read_excel(combine_78516)
combine_training_data = combine_training_data.append(df_78516, ignore_index=True)

combine_944601 = '/Users/simonli/Desktop/data297/fomc-hawkish-dovish-main/training_data/test-and-training/training_data/lab-manual-combine-train-944601.xlsx'
df_944601 = pd.read_excel(combine_944601)
combine_training_data = combine_training_data.append(df_944601, ignore_index=True)


/var/folders/fr/0_7xr4pj1yb9zdvs7vk07dd80000gn/T/ipykernel_4910/2953206703.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine_training_data = combine_training_data.append(df_5768, ignore_index=True)
/var/folders/fr/0_7xr4pj1yb9zdvs7vk07dd80000gn/T/ipykernel_4910/2953206703.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine_training_data = combine_training_data.append(df_78516, ignore_index=True)
/var/folders/fr/0_7xr4pj1yb9zdvs7vk07dd80000gn/T/ipykernel_4910/2953206703.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine_training_data = combine_training_data.append(df_944601, ignore_index=True)


In [5]:
#check random sample some data from the combine_training_data
combine_training_data.sample(10)

,index,sentence,year,label
5323,551,"More subtly, my conclusion that the effects on...",2002,2
3276,652,Participants noted that the improved performan...,2011,2
730,739,Some measures of inflation expectations were d...,2018,0
980,796,The role of the Board vis-à-vis the regional B...,2020,2
5491,810,The tendency may be natural to allow more flex...,2021,0
72,840,These inferences are supported by some empiric...,2001,2
2728,557,"Most participants expected that, following the...",2007,2
417,432,Investor perceptions of a somewhat less accomm...,2020,0
5495,567,My colleagues and I are acutely aware that hig...,2013,1
3983,530,"Moreover, anecdotal evidence suggested that mo...",2003,1


In [6]:
#extract the text and label from the combine_training_data
X = combine_training_data['sentence']
y = combine_training_data['label']

In [7]:
#check the label distribution
print(combine_training_data['label'].value_counts())

2    2773
0    1504
1    1432
Name: label, dtype: int64


In [8]:
X = preprocess(X)
y = np.array(y)

In [9]:
#check the shape of X and y
print(X.shape)
print(y.shape)

(5709, 100)
(5709,)


In [10]:
#load the test data
combine_test_data = pd.DataFrame()

# Read an Excel file using pandas
combine_5768_test = '/Users/simonli/Desktop/data297/fomc-hawkish-dovish-main/training_data/test-and-training/test_data/lab-manual-combine-test-5768.xlsx'
df_5768_test = pd.read_excel(combine_5768_test)
combine_test_data = combine_test_data.append(df_5768_test, ignore_index=True)

combine_78516_test = '/Users/simonli/Desktop/data297/fomc-hawkish-dovish-main/training_data/test-and-training/test_data/lab-manual-combine-test-78516.xlsx'
df_78516_test = pd.read_excel(combine_78516_test)
combine_test_data = combine_test_data.append(df_78516_test, ignore_index=True)

combine_944601_test = '/Users/simonli/Desktop/data297/fomc-hawkish-dovish-main/training_data/test-and-training/test_data/lab-manual-combine-test-944601.xlsx'
df_944601_test = pd.read_excel(combine_944601_test)
combine_test_data = combine_test_data.append(df_944601_test, ignore_index=True)

X_test = combine_test_data['sentence']
y_test = combine_test_data['label']

X_test = preprocess(X_test)
y_test = np.array(y_test)

/var/folders/fr/0_7xr4pj1yb9zdvs7vk07dd80000gn/T/ipykernel_4910/709729510.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine_test_data = combine_test_data.append(df_5768_test, ignore_index=True)
/var/folders/fr/0_7xr4pj1yb9zdvs7vk07dd80000gn/T/ipykernel_4910/709729510.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine_test_data = combine_test_data.append(df_78516_test, ignore_index=True)
/var/folders/fr/0_7xr4pj1yb9zdvs7vk07dd80000gn/T/ipykernel_4910/709729510.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combine_test_data = combine_test_data.append(df_944601_test, ignore_index=True)


In [12]:
max_words = 5000
max_len = 100
# Build the LSTM model
embedding_dim = 50  # Adjust based on your embedding dimension
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
model.add(LSTM(units=64, activation='tanh'))
model.add(Dense(units=3, activation='softmax'))  # Assuming 3 classes (dovish, hawkish, neutral)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
# Train the model
epochs = [10,20,30]
batch_sizes = [4,8,16,32]

#create an dictionary to store result
result = {}
for epoch in epochs:
    for batch_size in batch_sizes:
        model.fit(X, y, epochs=epoch, batch_size=batch_size, validation_split=0.2)
        # Evaluate the model on the test set
        loss, accuracy = model.evaluate(X_test, y_test)
        #print test loss and accuracy with corresponding epoch and batch size
        print("Epoch: ", epoch, "Batch size: ", batch_size, "Test loss: ", loss, "Test accuracy: ", accuracy,"F1:", 2*accuracy*loss/(accuracy+loss))
        #store the result in the dictionary
        result[(epoch,batch_size)] = accuracy


Epoch 1/10
1142/1142 [==============================] - 30s 25ms/step - loss: 1.0560 - accuracy: 0.4826 - val_loss: 1.0478 - val_accuracy: 0.4965
Epoch 2/10
1142/1142 [==============================] - 28s 25ms/step - loss: 1.0524 - accuracy: 0.4839 - val_loss: 1.0417 - val_accuracy: 0.4974
Epoch 3/10
1142/1142 [==============================] - 29s 25ms/step - loss: 1.0513 - accuracy: 0.4850 - val_loss: 1.0410 - val_accuracy: 0.4982
Epoch 4/10
1142/1142 [==============================] - 29s 25ms/step - loss: 1.0487 - accuracy: 0.4854 - val_loss: 1.0422 - val_accuracy: 0.4974
Epoch 5/10
1142/1142 [==============================] - 28s 24ms/step - loss: 1.0479 - accuracy: 0.4852 - val_loss: 1.0431 - val_accuracy: 0.4982
Epoch 6/10
1142/1142 [==============================] - 33s 29ms/step - loss: 1.0469 - accuracy: 0.4861 - val_loss: 1.0392 - val_accuracy: 0.4982
Epoch 7/10
1142/1142 [==============================] - 29s 25ms/step - loss: 1.0450 - accuracy: 0.4857 - val_loss: 1.0403 -

In [14]:
#print the result with highest accuracy
print("The result with highest accuracy is: ", max(result, key=result.get))
print(result)

The result with highest accuracy is:  (10, 8)
{(10, 4): 0.4831932783126831, (10, 8): 0.4838935434818268, (10, 16): 0.37745097279548645, (10, 32): 0.37254902720451355, (20, 4): 0.35574230551719666, (20, 8): 0.3522408902645111, (20, 16): 0.3445378243923187, (20, 32): 0.35574230551719666, (30, 4): 0.3263305425643921, (30, 8): 0.34663864970207214, (30, 16): 0.33263304829597473, (30, 32): 0.33473390340805054}
